In [13]:
import pandas as pd
import sqlalchemy as sql


In [14]:
server = "thesis.ca6j6heoraog.eu-central-1.rds.amazonaws.com"
engine = sql.create_engine(f"mysql+pymysql://admin:FphvsYQek4@{server}/thesis_db")


In [9]:
pers = pd.read_excel("../../data/shahar S data 4.3.2020.xlsx").drop("Unnamed: 5",axis=1)

In [15]:
con = engine.connect()

In [10]:
pers.describe()

,ID,Group number,condition,age,geder,trust,group_efficacy,similarity,group_identification,rapport,NewCohesion
count,150.000000,150.000000,150.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000
mean,179.400000,1026.420000,0.480000,22.536913,1.791946,4.970738,4.355705,4.048322,6.257450,6.257450,4.070470
std,45.065798,14.609122,0.501274,2.083635,0.407284,0.941527,1.004711,0.729921,1.773268,1.773268,1.047471
min,101.000000,1001.000000,0.000000,18.000000,1.000000,2.110000,2.000000,1.900000,1.000000,1.000000,1.000000
25%,141.250000,1014.000000,0.000000,21.000000,2.000000,4.330000,3.750000,3.600000,5.000000,5.000000,3.500000
50%,178.500000,1026.500000,0.000000,22.000000,2.000000,4.890000,4.250000,4.000000,6.330000,6.330000,4.250000
75%,218.750000,1039.000000,1.000000,24.000000,2.000000,5.670000,5.000000,4.500000,7.670000,7.670000,4.750000
max,256.000000,1051.000000,1.000000,32.000000,2.000000,7.000000,7.000000,6.000000,9.000000,9.000000,6.000000


In [11]:
pers.head()

,ID,Group number,condition,age,geder,trust,group_efficacy,similarity,group_identification,rapport,NewCohesion
0,101,1001,0,25.0,1.0,4.89,5.25,4.3,6.67,6.67,4.25
1,102,1001,0,21.0,2.0,6.00,5.50,4.4,7.33,7.33,5.00
2,103,1001,0,23.0,2.0,6.44,5.75,4.9,9.00,9.00,5.25
3,104,1002,1,22.0,2.0,4.78,3.25,4.0,4.67,4.67,3.75
4,105,1002,1,23.0,2.0,5.11,3.50,4.5,7.00,7.00,4.50


In [16]:
pars = pd.read_sql_table("participants",con)

In [17]:
pars.head()

,participant_id,group_id
0,101,1001
1,102,1001
2,103,1001
3,108,1003
4,109,1003


In [20]:
pd.merge(pars, pers[["ID","Group number"]], 
         how='outer', 
         left_on =["participant_id","group_id"], 
         right_on =["ID","Group number"]
        ).sort_values(["group_id","participant_id"]).to_html("id_compare.html")

In [25]:
con.execute("""
ALTER TABLE participants
ADD COLUMN cohesion decimal(11,5) """)

In [28]:
quer = """
UPDATE participants
SET cohesion = :coh
WHERE participant_id = :pid
"""

for row in pers.itertuples():
    print(quer.replace(":coh",str(row.NewCohesion)).replace(":pid",str(row.ID)))
    break


UPDATE participants
SET cohesion = 4.25
WHERE participant_id = 101



In [34]:
quer = """
UPDATE participants
SET cohesion = :coh
WHERE participant_id = :pid
"""

for row in pers[~pers.NewCohesion.isna()].itertuples():
    #con.execute(quer, coh= row.NewCohesion, pid=row.ID)
    con.execute(quer.replace(":coh",str(row.NewCohesion)).replace(":pid",str(row.ID)))

In [23]:
pers.dtypes

ID                        int64
Group number              int64
condition                 int64
age                     float64
geder                   float64
trust                   float64
group_efficacy          float64
similarity              float64
group_identification    float64
rapport                 float64
NewCohesion             float64
dtype: object